# emTools

### a Python package to interact with SerialEM to enable automated Transmission Electron Microscopy

## Installation instructions

- Install Anaconda from [here](https://www.anaconda.com/download/ "Download Anaconda")
- Start the Anaconda [Navigator](http://docs.anaconda.com/anaconda/user-guide/getting-started/ "Getting started with Anaconda")
- as explained in [section 4](http://docs.anaconda.com/anaconda/user-guide/getting-started/#run-python-in-a-jupyter-notebook "Run Python in a Jupyter Notebook"), install and start a Jupyter notebook.
- Download this [tutorial file] and follow the instructions.
- in the Anaconda terminal run: conda install -c conda-forge tifffile numpy scipy pandas scikit-image git
- then run pip install git+https://git.embl.de/schorb/serialem_tools


In [5]:
import conda.cli

conda.cli.main('conda', 'install',  '-y', 'numpy', 'scipy', 'pandas', 'scikit-image', 'git')
conda.cli.main('conda', 'install',  '-y', '-c conda-forge', 'tifffile')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done


ca-certificates-2018 | 135 KB    |            |   0% 


## Package Plan ##

  environment location: /g/emcf/software/python/anaconda2

  added / updated specs: 
    - tifffile


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    future-0.17.0              |        py27_1000         708 KB  conda-forge
    conda-4.5.11               |        py27_1000         651 KB  conda-forge
    tifffile-0.15.1            |   py27h7eb728f_1         206 KB  conda-forge
    backports.lzma-0.0.13      |   py27h8fe5b11_0          47 KB  conda-forge
    certifi-2018.10.15         |        py27_1000         138 KB  conda-forge
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    pathlib-1.0.1              |             py_1          15 KB  conda-forge
    ------------------------------------------------------------
                                           T

ca-certificates-2018 | 135 KB    | ########## | 100% 
future-0.17.0        | 708 KB    | ########## | 100% 
conda-4.5.11         | 651 KB    | ########## | 100% 
tifffile-0.15.1      | 206 KB    | ########## | 100% 
backports.lzma-0.0.1 | 47 KB     | ########## | 100% 
certifi-2018.10.15   | 138 KB    | ########## | 100% 
openssl-1.0.2p       | 3.1 MB    | ########## | 100% 
pathlib-1.0.1        | 15 KB     | ########## | 100% 

Preparing transaction: ...working... 

done
Verifying transaction: ...working... done
Executing transaction: ...working... done
